# Candidate tagging front end
## Quick way to add string to candidate for filtering

In [ ]:
import rflearn.elastic
import os.path
from numpy import argsort
from IPython.display import Image, display
from ipywidgets import Text

In [ ]:
# core functions

def serveimage(imagename, baseurl='http://www.aoc.nrao.edu/~claw/plots/', width=700):
    display(Image(url=os.path.join(baseurl, imagename), width=width))
    
def gettag(tag):
    return tag

def tagloop(datalist, order):
    """ Generator for tagged data. order is result of argsort on some candidate property """
    
    for candi in order:
        data = datalist[candi]
        serveimage(data['candidate_png'])
        print('Current tag: {0}'.format(data['tag']))

# tried to get widget to work, but they don't work well in loop
#        tag = Text(value=data['tag'])
#        hndl = display(tag)
#        more = raw_input("Any key to continue...")
#        data['tag'] = tag.value

        tag = raw_input("Set new tag list (or enter to keep old tag):")
        if tag:
            data['tag'] = tag
            print('New tag: {0}'.format(data['tag']))

        yield data

In [ ]:
# read from elastic
datalist = rflearn.elastic.indextodatalist()

# optionally filter datalist here...
datalist = [data for data in datalist if not data['tag']]

In [ ]:
# define iteration order
snr = [data['snr1'] for data in datalist]
rbscore = [data['rbscore'] for data in datalist]

# run tagging loop
loop = tagloop(datalist, argsort(snr)[::-1])
datalist2 = []
while True:
    try:
        data = loop.next()
        datalist2.append(data)
    except KeyboardInterrupt:
        print('Exiting loop..')
        break
        
rflearn.elastic.pushdata(datalist2)